Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na "YOUR ANSWER HERE" ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
Eto ilay references rehetra no apetraka

---https://github.com/lx10077/lasso/blob/master/project_gradient.py
https://xavierbourretsicotte.github.io/lasso_implementation.html

In [1]:
from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import normalize


def grad_check_sparse(f, x, analytic_grad, num_checks=10, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
data = load_diabetes()
X, y = data.data, data.target

In [3]:
def mse_loss_vectorized(w, b, X, y):
    """
    MSE loss function WITHOUT FOR LOOPs , NO REGULARIZATION
    
    Returns a tuple of:
    - loss 
    - gradient with respect to weights w
    - gradient with respect to bias b
    """
    loss = 0.0
    dw = np.zeros_like(w)
    
    # YOUR CODE HERE
    n = len(y)
    ## we don't put regularization here 
    loss = np.transpose((X @ w + b) - y) @ ((X @ w + b) - y)/n
    dw = 2*(np.transpose(X) @ (X @ w - y +b))/n
    db = (2*(((X @ w) - y) + b).sum())/n
    
    return loss, dw, np.array(db).reshape(1,)

In [4]:
def soft_threshold(x, delta):
    # YOUR CODE HERE
    if x - delta > 0 :
        return x - delta
    elif x + delta < 0 :
        return x + delta
    else:
        return 0

# Lasso Subgradient Descent

In [5]:
def l1_subgradient(w):
    """
    Subgradient of the L1 loss
    """
    dw = np.zeros_like(w)
    dw = np.sum([np.sign(i) for i in w ])
    
    return dw
    

def lasso_subgradient_mse_loss_vectorized(w, b, X, y, alpha):
    """
    MSE loss function adding the subgradient for w
    """
    loss, dw, db = mse_loss_vectorized(w, b, X, y)
    # Add the subgradient to dw
    dw += alpha * l1_subgradient(w)
    
    return loss, dw, db

In [6]:
class LassolSubgradientDescent():
    def __init__(self,  alpha=0.1):
        self.w = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            i = np.random.choice (N, batch_size)
            ## building matrix X and vector y
            X_batch = X[i, :]
            y_batch = y[i]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            self.w = self.w - learning_rate * dw
            self.b = self.b - learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
    
    
    def predict(self, X):
        return np.dot(X, self.w) + self.b

    def loss(self, X_batch, y_batch):
        return lasso_subgradient_mse_loss_vectorized(self.w, self.b, X_batch, y_batch, self.alpha)

In [7]:
model = LassolSubgradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse = mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 32194.630213
iteration 10000 / 200000: loss 3406.145285
iteration 20000 / 200000: loss 3641.027538
iteration 30000 / 200000: loss 3262.964564
iteration 40000 / 200000: loss 2903.784041
iteration 50000 / 200000: loss 2796.174114
iteration 60000 / 200000: loss 2745.275034
iteration 70000 / 200000: loss 2792.628683
iteration 80000 / 200000: loss 2956.126997
iteration 90000 / 200000: loss 2676.507070
iteration 100000 / 200000: loss 3116.362864
iteration 110000 / 200000: loss 2959.973704
iteration 120000 / 200000: loss 2570.398162
iteration 130000 / 200000: loss 2570.244815
iteration 140000 / 200000: loss 2535.985870
iteration 150000 / 200000: loss 2691.279906
iteration 160000 / 200000: loss 2673.714608
iteration 170000 / 200000: loss 3210.256156
iteration 180000 / 200000: loss 2359.187152
iteration 190000 / 200000: loss 3430.127294
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2881.5164692140074


In [8]:
model = LassolSubgradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse = mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 29290.106178
iteration 10000 / 200000: loss 3460.690006
iteration 20000 / 200000: loss 3454.922802
iteration 30000 / 200000: loss 2769.743136
iteration 40000 / 200000: loss 2635.245184
iteration 50000 / 200000: loss 2769.347402
iteration 60000 / 200000: loss 2279.575753
iteration 70000 / 200000: loss 2875.926954
iteration 80000 / 200000: loss 2620.704933
iteration 90000 / 200000: loss 2965.676871
iteration 100000 / 200000: loss 2929.711338
iteration 110000 / 200000: loss 2965.600097
iteration 120000 / 200000: loss 3103.809192
iteration 130000 / 200000: loss 2760.247116
iteration 140000 / 200000: loss 2554.624774
iteration 150000 / 200000: loss 3103.765484
iteration 160000 / 200000: loss 2958.661678
iteration 170000 / 200000: loss 3061.904776
iteration 180000 / 200000: loss 3137.866490
iteration 190000 / 200000: loss 3200.264530
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 2888.4419348900565


# Lasso Proximal Gradient Descent (iterative soft thresholding algorithm or ISTA)

In [9]:
class LassoProximalGradientDescent():
    def __init__(self,  alpha=0.1):
        self.w = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            i = np.random.choice (N, batch_size)
            ## building matrix X and vector y
            X_batch = X[i, :]
            y_batch = y[i]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # PROJECT THE GRADIENT FOR w USING soft_threshold
            for i in range(dw.shape[0]):
                self.w[i] = soft_threshold(self.w[i] - learning_rate * dw[i], learning_rate * self.alpha)
            self.b = self.b - learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

    def predict(self, X):
        return np.dot(X, self.w) + self.b

    def loss(self, X_batch, y_batch):
        return mse_loss_vectorized(self.w, self.b, X_batch, y_batch)

In [10]:
model = LassoProximalGradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 26176.646898
iteration 10000 / 200000: loss 3543.763488
iteration 20000 / 200000: loss 3133.983543
iteration 30000 / 200000: loss 3589.058392
iteration 40000 / 200000: loss 2839.291389
iteration 50000 / 200000: loss 2881.438408
iteration 60000 / 200000: loss 2589.463789
iteration 70000 / 200000: loss 3056.928897
iteration 80000 / 200000: loss 2837.529702
iteration 90000 / 200000: loss 2902.390264
iteration 100000 / 200000: loss 3241.357240
iteration 110000 / 200000: loss 3162.780850
iteration 120000 / 200000: loss 3040.279811
iteration 130000 / 200000: loss 3097.119286
iteration 140000 / 200000: loss 2951.008413
iteration 150000 / 200000: loss 2335.299645
iteration 160000 / 200000: loss 2860.160314
iteration 170000 / 200000: loss 2794.724481
iteration 180000 / 200000: loss 3191.096757
iteration 190000 / 200000: loss 2680.070098
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2888.7374590635036


In [11]:
model = LassoProximalGradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 30233.788824
iteration 10000 / 200000: loss 4403.104944
iteration 20000 / 200000: loss 4586.764681
iteration 30000 / 200000: loss 4007.653908
iteration 40000 / 200000: loss 4291.985922
iteration 50000 / 200000: loss 3605.601848
iteration 60000 / 200000: loss 3696.168961
iteration 70000 / 200000: loss 4110.984242
iteration 80000 / 200000: loss 4065.008901
iteration 90000 / 200000: loss 3949.386818
iteration 100000 / 200000: loss 3866.414453
iteration 110000 / 200000: loss 3418.801779
iteration 120000 / 200000: loss 4064.773149
iteration 130000 / 200000: loss 3894.589542
iteration 140000 / 200000: loss 3442.405646
iteration 150000 / 200000: loss 3807.173251
iteration 160000 / 200000: loss 4573.922115
iteration 170000 / 200000: loss 3843.138974
iteration 180000 / 200000: loss 3528.614825
iteration 190000 / 200000: loss 3487.396665
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 3812.138322912674


# Lasso Projected Gradient Descent

In [12]:
class LassoProjectedGradientDescent():
    def __init__(self,  alpha=0.1):
        self.w_p = None
        self.w_n = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w_p is None: # Initialization
            self.w_p = 0.001 * np.random.randn(d)
            self.w_n = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            i = np.random.choice (N, batch_size)
            ## building matrix X and vector y
            X_batch = X[i, :]
            y_batch = y[i]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # Project for w_p and w_n
            
            self.w_p -= learning_rate * (dw + self.alpha)
            self.w_n -= learning_rate * (-dw + self.alpha)
            self.w_p[self.w_p <= 0.] = 0.
            self.w_n[self.w_n <= 0.] = 0.
            self.b = self.b - learning_rate*db

            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

    def predict(self, X):
        return np.dot(X, self.w_p - self.w_n) + self.b

    def loss(self, X_batch, y_batch):
        return mse_loss_vectorized((self.w_p - self.w_n), self.b, X_batch, y_batch)

In [13]:
model = LassoProjectedGradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 26792.042948
iteration 10000 / 200000: loss 3517.719509
iteration 20000 / 200000: loss 2860.556697
iteration 30000 / 200000: loss 2831.803223
iteration 40000 / 200000: loss 3037.089705
iteration 50000 / 200000: loss 3125.155428
iteration 60000 / 200000: loss 3142.537072
iteration 70000 / 200000: loss 2688.109726
iteration 80000 / 200000: loss 2986.825662
iteration 90000 / 200000: loss 3039.440740
iteration 100000 / 200000: loss 3408.174265
iteration 110000 / 200000: loss 2714.012636
iteration 120000 / 200000: loss 2893.340629
iteration 130000 / 200000: loss 2952.892501
iteration 140000 / 200000: loss 2622.258448
iteration 150000 / 200000: loss 3036.095453
iteration 160000 / 200000: loss 2673.879405
iteration 170000 / 200000: loss 3242.664005
iteration 180000 / 200000: loss 3112.369008
iteration 190000 / 200000: loss 3154.646766
MSE scikit-learn: 2912.521795117546
MSE Coordinate descent model : 2888.8854898919344


In [14]:
model = LassoProjectedGradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 30248.102740
iteration 10000 / 200000: loss 4663.339717
iteration 20000 / 200000: loss 4013.533590
iteration 30000 / 200000: loss 4051.102263
iteration 40000 / 200000: loss 3672.301397
iteration 50000 / 200000: loss 3475.679146
iteration 60000 / 200000: loss 4190.293050
iteration 70000 / 200000: loss 3782.279509
iteration 80000 / 200000: loss 3757.703645
iteration 90000 / 200000: loss 3554.159927
iteration 100000 / 200000: loss 3985.342751
iteration 110000 / 200000: loss 3320.630785
iteration 120000 / 200000: loss 3801.441435
iteration 130000 / 200000: loss 3779.767082
iteration 140000 / 200000: loss 3993.868534
iteration 150000 / 200000: loss 3969.171923
iteration 160000 / 200000: loss 3383.587168
iteration 170000 / 200000: loss 3485.816860
iteration 180000 / 200000: loss 4071.309137
iteration 190000 / 200000: loss 3238.134366
MSE scikit-learn: 5650.287416333697
MSE Coordinate descent model : 3812.163086627614


# Lasso Coordinate Descent (no intercept)

In [15]:
class LassoCoordinateDescent():
    def __init__(self, alpha=0.1):
        self.w = None
        self.alpha = alpha

    def train(self, X, y, num_iters=1000):
        N, d = X.shape
        
        ## First we initialize
        if self.w is None:
            self.w = 0.0001 * np.random.randn(d)
        for i in range(num_iters):
            for j in range(d):
                xj = X[:,j]
                y_pred = X @ self.w 
                self.w[j] = soft_threshold(xj.T @ (y - y_pred + self.w[j] * xj),self.alpha)
                
    def predict(self, X): 
        return np.dot(X, self.w)

In [16]:
model = LassoCoordinateDescent(alpha=0.1)
model.train(X, y)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=False)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

MSE scikit-learn: 26057.118798659812
MSE Coordinate descent model : 26004.297709341427


In [17]:
model = LassoCoordinateDescent(alpha=2)
model.train(X, y)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=False)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

MSE scikit-learn: 28794.88441987604
MSE Coordinate descent model : 26006.416566913213
